In [5]:
## if 1st time uncomment below installation 
# !pip install -U sentence-transformers
# !pip install langchain openai python-dotenv
# !pip install psycopg2-binary pgvector
# !pip install -U langchain-community
# !pip install fastapi
# !pip install uvicorn



## imoprting all the neccessary  files 
from sentence_transformers import SentenceTransformer
from langchain.document_loaders import TextLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import psycopg2
from dotenv import load_dotenv
import os
from fastapi import FastAPI
app =  FastAPI()

In [2]:
#
load_dotenv()

embeddingsmodel = SentenceTransformer("all-MiniLM-L6-v2")

def getembeddings(sentences, print_flag=True):
    # Sentences are encoded by calling model.encode()
    embeddings = embeddingsmodel.encode(sentences)

    if print_flag:
        # Print the embeddings
        for sentence, embedding in zip(sentences, embeddings):
            print("Sentence:", sentence)
            # Print a summary of the embedding instead of the full embedding
            print("Embedding", embedding)
            print("")

    return embeddings


#  database connection details
host = os.getenv('DB_HOST')
dbname = os.getenv('DB_NAME')
user =  os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')

try:
    connection = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)
    print("Database connection established.")
except Exception as e:
    print(f"Failed to connect to the database: {e}")


/home/illahi/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Database connection established.


In [11]:
# loader = TextLoader('state_of_the_union.txt', encoding='utf-8')
file_location = '/home/illahi/Downloads/Matthew.pdf'

if file_location.endswith('.txt'):
    # Create a loader instance for the text file
    loader = TextLoader(file_location, encoding='utf-8')
    documents = loader.load()
else:
    # Create a loader instance for the PDF file
    loader = PyPDFLoader(file_location)
    documents = loader.load_and_split()
    print(documents[0])

print(documents)

# print(documents)  # prints the document objects
# print(len(documents))  # 1 - we've only read one file/document into the loader

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_documents(documents)

print(t.page_content for t in texts[0])
print(type(texts[0]))
print

page_content='Lorem ipsum \nLorem ipsum dolor sit amet, consectetur adipiscing \nelit. Nunc ac faucibus odio. \nVestibulum neque massa, scelerisque sit amet ligula eu, congue molestie mi. Praesent ut\nvarius sem. Nullam at porttitor arcu, nec lacinia nisi. Ut ac dolor vitae odio interdum\ncondimentum.  Vivamus  dapibus  sodales  ex,  vitae  malesuada  ipsum  cursus\nconvallis. Maecenas sed egestas nulla, ac condimentum orci.  Mauris diam felis,\nvulputate ac suscipit et, iaculis non est. Curabitur semper arcu ac ligula semper, nec luctus\nnisl blandit. Integer lacinia ante ac libero lobortis imperdiet. Nullam mollis convallis ipsum,\nac accumsan nunc vehicula vitae. Nulla eget justo in felis tristique fringilla. Morbi sit amet\ntortor quis risus auctor condimentum. Morbi in ullamcorper elit. Nulla iaculis tellus sit amet\nmauris tempus fringilla.\nMaecenas mauris lectus, lobortis et purus mattis, blandit dictum tellus.\n\uf0b7Maecenas non lorem quis tellus placerat varius. \n\uf0b7Null

<function print>

In [4]:
doc_vectors = getembeddings([t.page_content for t in texts],False)
print(len(doc_vectors[0]))  # 5 vectors in the output
print(type(doc_vectors[0]))   

384
<class 'numpy.ndarray'>


In [6]:
with connection.cursor() as cursor:
    # Construct the SQL INSERT statement
    insert_statement = """
    INSERT INTO embeddings_table (text_data, embeddings) VALUES (%s, %s)
    """
    
    # Process each item
    for ind, (sentence, embedding) in enumerate(zip(texts, doc_vectors)):
        try:
            # Data to insert
            embedding_value = embedding.tolist()  # Convert numpy array to list
            text_value = sentence.page_content  # Make sure 'sentence' has the attribute 'page_content'
            
            # Execute the SQL command
            cursor.execute(insert_statement, (text_value, embedding_value))
            
        except Exception as e:
            print(f"Failed to insert record {ind}: {e}")
            connection.rollback()  # Rollback the transaction
            continue  # Continue with the next record
        
    connection.commit()  # Commit the transaction
#
connection.rollback()  # Rollback the transaction


In [12]:
prompt = "My plan will cut the cost in half for most families and help parents"

prompt_embedding = getembeddings(prompt,False)
prompt_embedding = prompt_embedding.tolist()
print(prompt_embedding)
# 

[-0.014311795122921467, 0.10121113806962967, 0.026896808296442032, -0.04116528853774071, 0.014222769066691399, -0.0023053258191794157, -0.026518169790506363, 0.0646955594420433, -0.022763807326555252, 0.03775046020746231, 0.05753549560904503, -0.0366654172539711, -0.05759327486157417, -0.0033083069138228893, 0.010693574324250221, 0.0033677113242447376, 0.012423682026565075, -0.004332914017140865, -0.07366693019866943, -0.029483433812856674, 0.03945855051279068, -0.009361464530229568, -0.025470387190580368, 0.05038619041442871, 0.07336822897195816, 0.06897926330566406, 0.03222593665122986, -0.03285686671733856, -0.016742927953600883, 0.01816936954855919, 0.12629134953022003, 0.007742286194115877, 0.014189207926392555, -0.027896033599972725, 0.023071015253663063, 0.04129417985677719, -0.024385875090956688, 0.030921587720513344, -0.024547072127461433, -0.008407274261116982, 0.016920529305934906, 0.06191729009151459, -0.04269830137491226, -0.024068092927336693, 0.01320003904402256, -0.0042

In [33]:
cursor = connection.cursor()


query = """
SELECT id, text_data, 1 - (embeddings <=> %s::vector) AS similarity
FROM embeddings_table
ORDER BY similarity DESC
LIMIT 1;
"""


cursor.execute(query, (prompt_embedding,))

results = cursor.fetchall()
texts = [text for _, text, _ in results]

print(texts)


['sociosqu ad litora torquent per conubia nostra, per inceptos himenaeos. Ut ullamcorper\njusto sapien, in cursus libero viverra eget. Vivamus auctor imperdiet urna, at pulvinar leo\nposuere laoreet. Suspendisse neque nisl, fringilla at iaculis scelerisque, ornare vel dolor. Ut\net  pulvinar  nunc.  Pellentesque  fringilla  mollis  efficitur.  Nullam  venenatis  commodo\nimperdiet. Morbi velit neque, semper quis lorem quis, efficitur dignissim ipsum. Ut ac lorem']
